In [6]:
# constanten
BASIC_PATH = '../Data/'
ALL_FILES = BASIC_PATH + '*.csv'

from math import *


usaStates = [
    "AL",
    "AK",
    "AZ",
    "AR",
    "CA",
    "CO",
    "CT",
    "DE",
    "FL",
    "GA",
    "HI",
    "ID",
    "IL",
    "IN",
    "IA",
    "KS",
    "KY",
    "LA",
    "ME",
    "MD",
    "MA",
    "MI",
    "MN",
    "MS",
    "MO",
    "MT",
    "NE",
    "NV",
    "NH",
    "NJ",
    "NM",
    "NY",
    "NC",
    "ND",
    "OH",
    "OK",
    "OR",
    "PA",
    "RI",
    "SC",
    "SD",
    "TN",
    "TX",
    "UT",
    "VT",
    "VA",
    "WA",
    "WV",
    "WI",
    "WY",
]

In [7]:
# zelf geschreven functies

import pandas as pd
import numpy as np
import glob as glob

def readAllFiles():
    files = glob.glob(ALL_FILES)
    frames = []

    for file in files:
        df = pd.read_csv(file, index_col = 0)
        frames.append(df)

    return pd.concat(frames)

def readOneFile(url):
    return pd.read_csv(url, index_col = 0)

def exportDfToCsvFiles(df):
    step = 1000000
    start = 0
    stop = step
    i = 0

    while start < len(df):
        if stop >= len(df):
            stop = len(df)
        fileName = BASIC_PATH +  'flights_2010_' + str(i) + '.csv'
        
        data = df.iloc[start:stop, 0:]
        data.to_csv( fileName, sep=',')

        start += step
        stop += step
        i += 1
        
def deleteWrongStates(df):
    print("Aantal records:", len(df))
    copy = df
    
    for el in copy.departure_state.unique():
        if(el not in usaStates):
            copy = copy.drop(copy[copy['departure_state'] == el].index)
    print("Aantal records na verwijderen foute vertrek staat:", len(copy))

    for el in copy.arrival_state.unique():
        if(el not in usaStates):
            copy = copy.drop(copy[copy['arrival_state'] == el].index)
    print("Aantal records na verwijderen foute aankomst staat:", len(copy))
          
    return copy

def convertColumnTypes(df):
    df.departure_schedule = df.departure_schedule.astype(int)
    df.departure_delay = df.departure_delay.astype(float)
    df.arrival_schedule = df.arrival_schedule.astype(int)
    df.arrival_delay = df.arrival_delay.astype(float)
    df.arrival_actual = df.arrival_actual.astype(int)
    df.departure_actual = df.departure_actual.astype(int)
    return df

def dropMoreAdvancedDuplicates(df):
    copy = df
    copy = copy.groupby(['date', 'airline', 'airline_code', 'departure_airport', 'departure_state', 'departure_lat', 'departure_lon', 'departure_schedule', 'arrival_airport', 'arrival_state', 'arrival_lat', 'arrival_lon', 'arrival_schedule']).mean().reset_index()
    copy = convertColumnTypes(copy)
    copy = copy.drop(['index'], axis=1) #remove old index
    return copy

def calcDistance(df):
    array = [];
    for index, row in df.iterrows():
        array.append(calcTheDistance(row))
    return array;

def calcTheDistance(el):
    slat = radians(float(el["arrival_lat"]))
    slon = radians(float(el["arrival_lon"]))
    elat = radians(float(el["departure_lat"]))
    elon = radians(float(el["departure_lon"]))
    return 6371.01 * acos(sin(slat)*sin(elat) + cos(slat)*cos(elat)*cos(slon - elon))

def calcTheTime(data):
    durations = []
    for i, row in data.iterrows():
        duration = calcDuration(row["departure_schedule"], row["arrival_schedule"])
        durations.append(duration)
    return durations

def getTotalMinutes(time):
    time_str = str(int(time))
    if len(time_str) == 4:
        hours = time_str[0:2]
    else:
        hours = time_str[0:1]
    hours = int(hours)
        
    minutes = int(time_str[-2:])
    totalMinutes = minutes + hours * 60
    return totalMinutes

def getStringTime(totalMinutes):
    hours = math.floor(totalMinutes / 60)
    minutes = totalMinutes - (hours * 60)
    return str(hours) + ":" + str(minutes)

def calcDuration(departure_schedule, arrival_schedule):
    departure = getTotalMinutes(departure_schedule)
    arrival = getTotalMinutes(arrival_schedule)
    if arrival <= departure:
        return (24 * 60) - (departure - arrival)
    else:
        return arrival - departure
    
def calcSpeeds(data):
    speeds = []
    for i, row in data.iterrows():
        speed = row["distance"] / (row["duration"] / 60)
        speeds.append(speed)
    return speed

def isNormalSpeed(speed, distance):
    return (speed > 450 and speed <=1100) or (speed / distance) >= 1 and (speed / distance) <= 2 and speed <= 1100;

def calcNormalSpeed(data):
    isNormalSpeeds = []
    for i, row in data.iterrows():
        isNormalSpeeds.append(isNormalSpeed(row["speed"], row["distance"]))
    return isNormalSpeeds;

def cleanData(df):
    print("Aantal records om te beginnen:", len(df))
    copy = df
    
    copy = copy.drop_duplicates()
    print("Aantal records na verwijderen van dubbels:", len(copy))
    
    copy = copy.reset_index()
    print("Aantal records na nieuwe index:", len(copy))
    
    copy = copy.dropna()
    print("Aantal records na verwijderen lege waarden:", len(copy))
    
    copy = deleteWrongStates(copy)
    copy = convertColumnTypes(copy)
    
    copy = dropMoreAdvancedDuplicates(copy)
    print("Aantal records na het verwijderen van de verborgen dubbels:", len(copy))
    
    distance = calcDistance(copy)

    copy['distance'] = distance
    
    durations = calcTheTime(copy)
    
    copy["duration"] = durations
    
    speeds = calcSpeeds(copy)
    
    copy["speed"] = speeds
    
    isNormalSpeeds = calcNormalSpeed(copy)
    
    copy["isNormalSpeed"] = isNormalSpeeds
    
    copy = copy[copy["isNormalSpeed"] == True]
    
    return copy

In [8]:
df = readAllFiles()
df2 = cleanData(df)

Aantal records om te beginnen: 10642028
Aantal records na verwijderen van dubbels: 10642028
Aantal records na nieuwe index: 10642028
Aantal records na verwijderen lege waarden: 10642028
Aantal records: 10642028
Aantal records na verwijderen foute vertrek staat: 10642028
Aantal records na verwijderen foute aankomst staat: 10642028
Aantal records na het verwijderen van de verborgen dubbels: 10642028


NameError: name 'isNormalSpeed' is not defined

In [31]:
# ML
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
enc = LabelEncoder();

X = df.loc[:,['date', 'airline', 'departure_airport', 'arrival_airport']]
X.date = enc.fit_transform(X.date)
X.airline = enc.fit_transform(X.airline)
X.departure_airport = enc.fit_transform(X.departure_airport)
X.arrival_airport = enc.fit_transform(X.arrival_airport)

y = df.loc[:,['arrival_delay']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

model = LinearRegression()
model.fit(X_train, y_train)

y_predict = model.predict(X_test)


# The coefficients
print('Coefficients: \n', model.coef_)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_predict))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.5f' % r2_score(y_test, y_predict))


Coefficients: 
 [[-0.00441251 -0.01723296 -0.00230018 -0.00156591]]
Mean squared error: 1329.10
Variance score: 0.00061
0.000613246714151


In [ ]:
# ML model 2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
enc = LabelEncoder();

X = df.loc[:,['date', 'airline', 'departure_airport', 'arrival_airport']]
X.date = enc.fit_transform(X.date)
X.airline = enc.fit_transform(X.airline)
X.departure_airport = enc.fit_transform(X.departure_airport)
X.arrival_airport = enc.fit_transform(X.arrival_airport)

y = df.loc[:,['arrival_delay']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

model = RandomForestRegressor(random_state=0)
model.fit(X_train, y_train)

y_predict = model.predict(X_test)

# The mean squared error
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_predict))

# Explained variance score: 1 is perfect prediction
print('Variance score: %.5f' % r2_score(y_test, y_predict))


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [28]:
exportDfToCsvFiles(df2)

In [ ]:
#creating a json file
df2.to_json('flights_2010_file_time_update_good_data.json', orient='records')

In [4]:
# Controleer of er records zijn die een vertrektijd hebben die vroeger is dan de aankomsttijd
# Omdat de aankomstdatum niet bijgehouden wordt, kan je niet zeker zijn of het de volgende dag is een een fout record
# Daarom controleren we ook nog of de geplande vliegtijd meer dan x aantal minuten te snel is
# We hebben geen extreme waardes gevonden en daarom geen records verwijderd
df2[(df2.departure_schedule > df2.arrival_schedule) & ((df2.arrival_delay - df2.departure_delay) < -80)]

,date,airline,airline_code,departure_airport,departure_state,departure_lat,departure_lon,departure_schedule,arrival_airport,arrival_state,arrival_lat,arrival_lon,arrival_schedule,departure_actual,departure_delay,arrival_actual,arrival_delay


In [15]:
#selecting SouthWest Airlines
df_southWest = df2.loc[df['airline'] == "WN"]
#selecting Alaska Airlines
df_alaska = df2.loc[df['airline'] == "AS"]
#selecting Hawaiian Airlines
df_hawaiian = df2.loc[df['airline'] == "HA"]

In [16]:
df_southWest = df_southWest[['date', 'departure_airport','departure_state',
                          'departure_lat', 'departure_lon',
                          'arrival_airport', 'arrival_state',
                          'arrival_lat', 'arrival_lon']]
df_alaska = df_alaska[['date', 'departure_airport','departure_state',
                          'departure_lat', 'departure_lon',
                          'arrival_airport', 'arrival_state',
                          'arrival_lat', 'arrival_lon']]
df_hawaiian = df_hawaiian[['date', 'departure_airport','departure_state',
                          'departure_lat', 'departure_lon',
                          'arrival_airport', 'arrival_state',
                          'arrival_lat', 'arrival_lon']]

In [29]:
#creating a json file for SouthWest Airlines
df_southWest.to_json('SouthWest_Airlines.json', orient='records')

In [30]:
#creating a json file for Alaska Airlines
df_southWest.to_json('Alaska_Airlines.json', orient='records')

In [31]:
#creating a json file for Hawaiian Airlines
df_southWest.to_json('Hawaiian_Airlines.json', orient='records')